# pgvector quickstart


### Import libraries and environment variables

In [1]:
from __future__ import annotations

import dotenv


# Reload the variables in your '.env' file (override the existing variables)
dotenv.load_dotenv("../../.env", override=True)

True

In [ ]:
import os
import sys
from dotenv import load_dotenv
import logging



sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..'))) # Add the parent directory to the path since we work with notebooks
from helper_functions import *
from evaluation.evalute_rag import *

from logging_utils import get_logger, global_log_config


global_log_config(
    log_level=logging.getLevelName("DEBUG"),
    json=False,
)


# # Load environment variables from a .env file
# load_dotenv()

# # Set the OpenAI API key environment variable
# os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

Logger set up with log level: 10


/Users/malcolm/.pyenv/versions/3.10.14/envs/rag_techniques3/lib/python3.10/site-packages/deepeval/__init__.py:45: UserWarning: You are using deepeval version 0.21.70, however version 1.1.3 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


<loguru.logger handlers=[(id=1, level=10, sink=stdout)]>

2024-08-29 18:00:54.960 | DEBUG    | ipykernel.kernelbase:execute_request - kernelbase.py:802 | {'header': {'msg_id': '43763dae-7e976f631dffd87956d91e4a_15809_37', 'msg_type': 'execute_reply', 'username': 'malcolm', 'session': '43763dae-7e976f631dffd87956d91e4a', 'date': datetime.datetime(2024, 8, 29, 22, 0, 54, 960431, tzinfo=datetime.timezone.utc), 'version': '5.3'}, 'msg_id': '43763dae-7e976f631dffd87956d91e4a_15809_37', 'msg_type': 'execute_reply', 'parent_header': {'date': datetime.datetime(2024, 8, 29, 22, 0, 53, 725000, tzinfo=tzutc()), 'msg_id': 'f4666643-c4e0-45eb-bf34-dd97ca08ac7f', 'msg_type': 'execute_request', 'session': '11abcf1d-6b84-4ea8-aed8-3562b139c4b3', 'username': '0d7d68c9-732a-4d67-8718-c95829b84087', 'version': '5.2'}, 'content': {'status': 'ok', 'execution_count': 2, 'user_expressions': {}, 'payload': []}, 'metadata': {'started': datetime.datetime(2024, 8, 29, 22, 0, 53, 728037, tzinfo=datetime.timezone.utc), 'dependencies_met': True, 'engine': 'cff99ef5-ab07-4

### Read Docs

In [3]:
# SOURCE: https://www.youtube.com/watch?v=Ff3tJ4pJEa4
import pinecone
from dotenv import load_dotenv
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.document_loaders import TextLoader
from langchain.vectorstores.pgvector import PGVector
from pgvector_service import PgvectorService
import os
import time

2024-08-29 18:00:57.537 | DEBUG    | ipykernel.kernelbase:dispatch_shell - kernelbase.py:419 | 
*** MESSAGE TYPE:execute_request*** | {}
2024-08-29 18:00:57.538 | DEBUG    | ipykernel.kernelbase:dispatch_shell - kernelbase.py:420 |    Content: {'silent': False, 'store_history': True, 'user_expressions': {}, 'allow_stdin': True, 'stop_on_error': False, 'code': '# SOURCE: https://www.youtube.com/watch?v=Ff3tJ4pJEa4\nimport pinecone\nfrom dotenv import load_dotenv\nfrom langchain.document_loaders import TextLoader\nfrom langchain.embeddings.openai import OpenAIEmbeddings\nfrom langchain.text_splitter import CharacterTextSplitter\nfrom langchain.vectorstores import Pinecone\nfrom langchain.document_loaders import TextLoader\nfrom langchain.vectorstores.pgvector import PGVector\nfrom pgvector_service import PgvectorService\nimport os\nimport time'}
   --->
    | {}
2024-08-29 18:00:57.538 | DEBUG    | ipykernel.kernelbase:dispatch_shell - kernelbase.py:429 | execute_request: {'header': {'da

# Load the documents


In [4]:
loader = TextLoader(
    "../../data/The Project Gutenberg eBook of A Christmas Carol in Prose; Being a Ghost Story of Christmas.txt"
)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

query = "The Project Gutenberg eBook of A Christmas Carol in Prose; Being a Ghost Story of Christmas"


"""
First, we compare to Pinecone, a managed vector store service.

"""

/Users/malcolm/.pyenv/versions/3.10.14/envs/rag_techniques3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


'\nFirst, we compare to Pinecone, a managed vector store service.\n\n'

# Create / Load the Pinecone index

In [7]:
# pinecone.init(
#     api_key=os.getenv("PINECONE_API_KEY"), environment=os.getenv("PINECONE_ENV")
# )

import os
from pinecone.control.pinecone import Pinecone
from pinecone import ServerlessSpec

pc: Pinecone = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)

index_name = "demo-index"
if index_name not in pc.list_indexes():
    pc.create_index(name=index_name, metric="cosine", dimension=1536, spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ))
    pinecone_docsearch = pc.from_documents(
        docs, embeddings, index_name="demo-index"
    )
else:
    pinecone_docsearch = pc.from_existing_index(index_name, embeddings)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'edb20f3119e635e20454e4d61d19daa3', 'Date': 'Thu, 29 Aug 2024 22:07:38 GMT', 'Server': 'Google Frontend', 'Content-Length': '200', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"INVALID_ARGUMENT","message":"Bad request: Your free plan does not support indexes in the us-west-2 region of aws. To create indexes in this region, upgrade your plan."},"status":400}


# Query the index with LanChain

In [ ]:
def run_query_pinecone(docsearch, query):
    docs = docsearch.similarity_search(query, k=4)
    result = docs[0].page_content
    return result


def calculate_average_execution_time(func, *args, **kwargs):
    total_execution_time = 0
    num_runs = 10
    for _ in range(num_runs):
        start_time = time.time()
        result = func(*args, **kwargs)  # Execute the function with its arguments
        end_time = time.time()
        execution_time = end_time - start_time
        total_execution_time += execution_time
    average_execution_time = round(total_execution_time / num_runs, 2)
    print(result)
    print(
        f"\nThe function took an average of {average_execution_time} seconds to execute."
    )
    return


calculate_average_execution_time(
    run_query_pinecone, docsearch=pinecone_docsearch, query=query
)


"""
Now, we compare to PGVector, an open source vector store service.

"""

# Create a PGVector Store

In [ ]:

"""
Donwload postgresql to run locally:
https://www.postgresql.org/download/

How to install the pgvector extension:
https://github.com/pgvector/pgvector

Fix common installation issues:
https://github.com/pgvector/pgvector?tab=readme-ov-file#installation-notes
"""

COLLECTION_NAME = "The Project Gutenberg eBook of A Christmas Carol in Prose"

CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver=os.environ.get("PGVECTOR_DRIVER", "psycopg3"),
    host=os.environ.get("PGVECTOR_HOST", "localhost"),
    port=int(os.environ.get("PGVECTOR_PORT", "6432")),
    database=os.environ.get("PGVECTOR_DATABASE", "langchain"),
    user=os.environ.get("PGVECTOR_USER", "langchain"),
    password=os.environ.get("PGVECTOR_PASSWORD", "langchain"),
)

# create the store
db = PGVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    pre_delete_collection=False,
)

# load the store
pgvector_docsearch = PGVector(
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    embedding_function=embeddings,
)


# Query the index with PGVector

In [ ]:

def run_query_pgvector(docsearch, query):
    docs = docsearch.similarity_search(query, k=4)
    result = docs[0].page_content
    return result


calculate_average_execution_time(
    run_query_pgvector, docsearch=pgvector_docsearch, query=query
)


# Add more collections to the database

In [ ]:

loader = TextLoader("../../data/The Project Gutenberg eBook of Romeo and Juliet.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
new_docs = text_splitter.split_documents(documents)


COLLECTION_NAME_2 = "The Project Gutenberg eBook of Romeo and Juliet"

db = PGVector.from_documents(
    embedding=embeddings,
    documents=new_docs,
    collection_name=COLLECTION_NAME_2,
    connection_string=CONNECTION_STRING,
    pre_delete_collection=False,
)

# Query the index with multiple collections


In [ ]:

pg = PgvectorService(CONNECTION_STRING)


def run_query_multi_pgvector(docsearch, query):
    docs = docsearch.custom_similarity_search_with_scores(query, k=4)
    result = docs[0][0].page_content
    print(result)


run_query_multi_pgvector(pg, query)

# --------------------------------------------------------------
# Delete the collection
# --------------------------------------------------------------
pg.delete_collection(COLLECTION_NAME)
pg.delete_collection(COLLECTION_NAME_2)

# --------------------------------------------------------------
# Update the collection
# --------------------------------------------------------------
pg.update_collection(docs=docs, collection_name=COLLECTION_NAME)
